# Running 'ellipse':

Read test image (this is the image formerly distributed with the IRAF software as dev$pix):

In [1]:
from astropy.io import fits
image = fits.open("../../test/data/M51.fits")
pixel_data = image[0].data

Create an instance of the Ellipse class, passing the numpy 2-D array with the pixel data as argument to the constructor:

In [2]:
from ellipse.ellipse import Ellipse

ellipse = Ellipse(pixel_data)

Finally, run the fit_image method, getting as result an instance of class IsophoteList.

Wait until all the output prints out, down to the SMA = 0 central intensity value.

In [3]:
isophote_list = ellipse.fit_image(verbose=True)

#
# Semi-    Isophote      Ellipticity   Position    Grad.  Data  Flag  Iter. Stop
# major      mean                       Angle       rel.                    code
# axis     intensity                               error
#(pixel)                               (degree)
#
  10.00     1087.19        0.098        67.76      0.193    60    0    20     0
  11.00     1045.31        0.134        73.17      0.157    64    0    10     0
  12.10      988.67        0.157        68.42      0.158    70    0    10     0
  13.31      884.66        0.022        52.72      0.490    83    0    50     2
  14.64      857.60        0.022        52.72      0.521    91    0     3     5
  16.11      961.28        0.403        52.72      0.313    77    0    50     2
  17.72      887.29        0.358        50.58      0.145    88    0    10     0
  19.49      756.85        0.179        35.12      0.110   110    0    10     0
  21.44      681.95        0.211        35.82      0.153   119    0    10     0
  23.58  

In [4]:
type(isophote_list)

ellipse.isophote.IsophoteList

### Running 'ellipse' in a finer-grained way:

We can fit individual ellipses as well, by just calling the 'fit_isophote' method in the same Ellipse instance (passing the semi-major axis length to the method):

In [5]:
isophote = ellipse.fit_isophote(20.)

from ellipse.isophote import print_header
print_header(verbose=True)
isophote.print(verbose=True)

#
# Semi-    Isophote      Ellipticity   Position    Grad.  Data  Flag  Iter. Stop
# major      mean                       Angle       rel.                    code
# axis     intensity                               error
#(pixel)                               (degree)
#
  20.00      734.53        0.186        34.44      0.115   113    0    13     0


Note that in this case we get an instance of class Isophote, not IsophoteList as before:

In [6]:
type(isophote)

ellipse.isophote.Isophote

Regarding the example above, by calling the 'fit_isophote' method in a simple 'for' loop with successive values for the semi-major axis length, one may be lead to think that it will emulate the behavior of the 'fit_image' method. However, that would not in general result in the best fits. The fitting algorithm is quite sensitive to the initial guesses (the starting ellipse geometry) and other fitting parameters such as the step used to move from a given ellipse to the next, the area integration method, finding the maximum acceptable semi-major axis length, and such. The 'fit_image' method takes care of handling these details in a transparent way for the user.

In the 'isophote' package, the class constructors and method calls accept a variety of parameters that can be used to customize and fine-tune the fit process. Here we give a few eaxamples; please consult the code documentation for a complete description. 

For instance, the fit algorithm is quite sensitive to the initial guesses for the X and Y position of the center of the galaxy on the frame. When using default values as in the examples above, the methods assume that the galaxy is exactly centered in the frame. The fit algorithm can also fail to properly converge if either the ellipticiy or the position angle of the semi-major axis are too way off the true values. To override that, we should initialize the Ellipse constructor with an instance of class Geometry. This class encapsulates all data and behavior associated with a given ellipse's geometry.

In [7]:
import numpy as np
from ellipse.geometry import Geometry

# geometry parameters
x0 = 256.
y0 = 256.
sma = 20.
eps = 0.2
pa = 35. / 180. / np.pi # 35 deg converted to radians
astep = 0.1
linear_growth = False

g = Geometry(x0, y0, sma, eps, pa, astep, linear_growth)
ellipse = Ellipse(pixel_data, geometry=g)

#from now on, it's the same as in examples above

To further break down the fit process, one could explictly work with the Sample and Fitter classes, as exemplified below.

In [9]:
from ellipse.sample import Sample
from ellipse.fitter import Fitter

sample = Sample(pixel_data, 40, geometry=g)
fitter = Fitter(sample)
isophote = fitter.fit()

In here, we initially create an instance of the Sample class. This instance encapsulates everything associated with a given elliptical path over the image. This includes not only the geometry information, but also the raw intensity samples extracted from the image, as well as associated photometric quantities.

The Sample instance is used to initialize an instance of the Fitter class. This class has a number of controls to help in tweaking the fit. The final result of the 'fit' method is an instance of class Isophote with the final, fitted values of the geometry parameters.

# Plotting results:

Import packages necessary for plotting:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Basic plot of intensities as a function of (semi-major axis length) ** 1/4:

In [ ]:
plt.scatter(isophote_list.sma**0.25, isophote_list.intens)

plt.xlabel('SMA**1/4')
plt.ylabel('INTENS')
plt.title("M51 profile")
plt.show()